In [1]:
from mlapp import GitHubApp
from IPython import display

In [2]:
app_id = 27079
#key_file_path = '/Users/hamelsmu/.ssh/hamel-python-app.2019-03-15.private-key.pem'
key_file_path = '/hamel-python-app.2019-03-15.private-key.pem'

## Get the app installation

The test installation automatically fetches the first installation the app is found on.  This app is only installed in one place, so this just fetches the installation made on the public `hamelsmu/example-github-app` repo. 

In [3]:
ghapp = GitHubApp(pem_path=key_file_path, app_id=app_id)
install = ghapp.get_test_installation()

## Interact With Issues

#### Create a new issue

In [4]:
issue = install.create_issue(owner='hamelsmu',
                             repository='example-github-app',
                             title='test issue', 
                             body='this is automatically created')

#### Comment on an issue

In [5]:
issue.create_comment('Wooo!  Its time to do some machine learning!')

<IssueComment [ml-auto-labeler[bot]]>

#### Add a label to an issue

In [6]:
issue.add_labels('AI-is-taking-over')

[<ShortLabel [AI-is-taking-over]>]

#### See the issue here

In [7]:
print(issue.html_url)

display.Markdown(f'[{issue.html_url}]({issue.html_url})')

https://github.com/hamelsmu/example-github-app/issues/13


[https://github.com/hamelsmu/example-github-app/issues/13](https://github.com/hamelsmu/example-github-app/issues/13)

## Extract Data

All issues from `kubeflow/kubeflow`

Get a list of issues which you can use to train models

In [8]:
issues = GitHubApp.unpack_issues(client=install, 
                                 owner='kubeflow',
                                 repo='kubeflow')

100%|██████████| 489/489 [02:20<00:00,  2.85it/s]


In [9]:
print(f'there are {len(issues)} issues with labels')

there are 372 issues with labels
